In [1]:
import os
import cv2
import dlib
import mysql.connector
import datetime
import glob

# Connect to the MySQL database
cnx = mysql.connector.connect(
    host='localhost',
    user='root',
    password='ayman',
    database='project'
)
cursor = cnx.cursor()

detector = dlib.get_frontal_face_detector()

def detect_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    return faces

def extract_face_data(image, face):
    x1, y1 = face.left(), face.top()
    x2, y2 = face.right(), face.bottom()

    # Extract face image data
    face_image = image[y1:y2, x1:x2]
    _, face_data = cv2.imencode('.jpg', face_image)

    return face_data.tobytes()

def insert_face_data(image_data, filename, date):
    # Specify the query to insert the image into the table
    query = "INSERT INTO new_table (personne, localisation, date) VALUES (%s, %s, %s)"

    # Specify the values to be inserted
    values = (image_data, filename, date)

    # Execute the query
    cursor.execute(query, values)

    # Commit the changes to the database
    cnx.commit()

def close_database_connection():
    # Close the cursor and database connection
    cursor.close()
    cnx.close()

videos_directory = 'C:/Users/aayma/OneDrive/Bureau/videos/videos'

# Get all video files in the directory
video_files = glob.glob(os.path.join(videos_directory, '*.mp4'))

for video_path in video_files:
    # Get the creation date of the video file
    video_creation_time = datetime.datetime.fromtimestamp(os.path.getctime(video_path))

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Iterate over the frames in the video
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        frame_count += 1

        # Calculate the timestamp for the current frame
        frame_timestamp = video_creation_time + datetime.timedelta(seconds=frame_count / cap.get(cv2.CAP_PROP_FPS))

        # Detect faces in the frame
        faces = detect_faces(frame)

        # Process each detected face
        for face_counter, face in enumerate(faces):
            # Extract face data
            face_data = extract_face_data(frame, face)

            # Insert face data into the database
            insert_face_data(face_data, f"frame_{face_counter}", frame_timestamp)

       # cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture
    cap.release()

# Close the windows
cv2.destroyAllWindows()

# Close the database connection
close_database_connection()



KeyboardInterrupt: 

In [2]:
import os
import cv2
import dlib
import mysql.connector
from datetime import datetime, timedelta
import random

# Connect to the MySQL database
cnx = mysql.connector.connect(
    host='localhost',
    user='root',
    password='ayman',
    database='project'
)
cursor = cnx.cursor()

detector = dlib.get_frontal_face_detector()

def detect_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    return faces

def extract_face_data(image, face):
    x1, y1 = face.left(), face.top()
    x2, y2 = face.right(), face.bottom()

    # Extract face image data
    face_image = image[y1:y2, x1:x2]
    face_bytes = cv2.imencode('.jpg', face_image)[1].tobytes()

    return face_bytes

def insert_face_data(image_data, filename, timestamp, location):
    # Specify the query to insert the image into the table
    query = "INSERT INTO new_table (personne, localisation, date) VALUES (%s, %s, %s)"

    # Format the timestamp
    formatted_timestamp = timestamp.strftime("%Y-%m-%d %H:%M:%S")

    # Specify the values to be inserted
    values = (image_data, location, formatted_timestamp)

    # Execute the query
    cursor.execute(query, values)

    # Commit the changes to the database
    cnx.commit()

def close_database_connection():
    # Close the cursor and database connection
    cursor.close()
    cnx.close()

folder_path = 'C:/Users/aayma/OneDrive/Bureau/videos/faces'

time_increment = timedelta(minutes=30)  # Temps initial
current_time = datetime(2023, 5, 23, 10, 30, 00)  # Temps de départ

locationsList = [
    'Municipalité principale de Sfax, Beb Bhar, Sfax',
    'Hôtel de Ville de Sfax, Avenue Habib Bourguiba, Sfax',
    'Palais de Justice de Sfax, Rue de la République, Sfax',
    # ... liste complète des localisations ...
    'Rue Salah Ben Youssef, Sfax'
]

# Iterate over the files in the folder
for filename in os.listdir(folder_path):
    try:
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image = cv2.imread(os.path.join(folder_path, filename))

            # Detect faces in the image
            faces = detect_faces(image)

            # Process each detected face
            for face_counter, face in enumerate(faces):
                # Extract face data
                face_data = extract_face_data(image, face)

                # Choose a location randomly
                location = random.choice(locationsList)

                # Insert face data into the database with incremented time and chosen location
                insert_face_data(face_data, filename, current_time, location)

                # Increment time for the next image
                current_time += time_increment

            print(f"Processed image: {filename}")

    except Exception as e:
        print(f"Error processing image {filename}: {e}")

# Close the database connection
close_database_connection()
